You have to work on the files:
*  [Books](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz)
*  [Book ratings](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz)
*  [Users](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz)
*  [Goodbooks books](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz)
*  [Goodbooks ratings](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz)

### Notes

1.    It is mandatory to use GitHub for developing the project.
1.    The project must be a jupyter notebook.
1.    There is no restriction on the libraries that can be used, nor on the Python version.
1.    To read those files, you need to use the `encoding = 'latin-1'` option.
1.    All questions on the project **must** be asked in a public channel on [Zulip](https://focs.zulipchat.com), otherwise no  answer will be given.

In [24]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
import pandas as pd
import numpy as np
import re

In [4]:
# importo i dataset specificando index_col o no?

In [8]:
users = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz', compression = 'gzip', escapechar = "\\", encoding = 'latin-1', sep = ';', dtype = {'User-ID': 'int',
                                                                                                                                                                'Location': 'string',
                                                                                                                                                                'Age': 'Int64'
                                                                                                                                                                })
users.shape

(278858, 3)

In [16]:
books = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz', compression = 'gzip', escapechar = "\\", encoding = 'latin-1', sep = ';', dtype = {'ISBN': 'string',
                                                                                                                                                                                  'Book-title': 'string',
                                                                                                                                                                                  'Book-Author': 'string',
                                                                                                                                                                                  'Year-Of-Publication': 'int',
                                                                                                                                                                                  'Publisher': 'string',
                                                                                                                                                                                  'Image-URL-S': 'string',
                                                                                                                                                                                  'Image-URL-M': 'string',
                                                                                                                                                                                  'Image-URL-L': 'string',
                                                                                                                                                                                 })
books.shape

(271359, 8)

In [44]:
book_ratings = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz', compression = 'gzip', encoding = 'latin-1', sep = ';', dtype = {'User-ID': 'Int64',
                                                                                                                                                                                                                 'ISBN': 'string',
                                                                                                                                                                                                                 'Book-Rating': 'Int64'
                                                                                                                                                                                                                })
book_ratings.shape

(1149780, 3)

In [45]:
goodbooks_ratings = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz', compression = 'gzip', encoding = 'latin-1', sep = ',', dtype = {'user_id': 'int',
                                                                                                                                                                                        'book_id': 'int',
                                                                                                                                                                                        'rating': 'int'
                                                                                                                                                                                        })
goodbooks_ratings.shape

(99, 3)

In [46]:
goodbooks = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz', compression = 'gzip', encoding = 'latin-1', sep = ',', dtype = {'book_id': 'int',
                                                                                                                                                                         'goodreads_book_id': 'int',
                                                                                                                                                                         'best_book_id': 'int',
                                                                                                                                                                         'work_id': 'int',
                                                                                                                                                                         'books_count': 'int',
                                                                                                                                                                         'isbn': 'string',
                                                                                                                                                                         'isbn13': 'string',
                                                                                                                                                                         'authors': 'string',
                                                                                                                                                                         'original_publication_year': 'int',
                                                                                                                                                                         'original_title': 'string',
                                                                                                                                                                         'title': 'string',
                                                                                                                                                                         'language_code': 'string',
                                                                                                                                                                         'average_rating': 'double',
                                                                                                                                                                         'ratings_count': 'int',
                                                                                                                                                                         'work_ratings_count': 'int',
                                                                                                                                                                         'work_text_reviews_count': 'int',
                                                                                                                                                                         'ratings_1': 'int',
                                                                                                                                                                         'ratings_2': 'int',
                                                                                                                                                                         'ratings_3': 'int',
                                                                                                                                                                         'ratings_4': 'int',
                                                                                                                                                                         'ratings_5': 'int',
                                                                                                                                                                         'image_url': 'string',
                                                                                                                                                                         'small_image_url': 'string'
                                                                                                                                                                         })
goodbooks.shape

(99, 23)

### 1. Normalize the location field of *Users* dataset, splitting into city, region, country.

In [43]:
users[['City', 'Region', 'Country']] = users['Location'].str.rsplit(',', 2, expand=True)
users.head()

# Da sistemare country usando le probabilitá (?)
# creare lista 
# sistemare encoding (riga 1318)

,User-ID,Location,Age,City,Region,Country
0,1,"nyc, new york, usa",<NA>,nyc,new york,usa
1,2,"stockton, california, usa",18,stockton,california,usa
2,3,"moscow, yukon territory, russia",<NA>,moscow,yukon territory,russia
3,4,"porto, v.n.gaia, portugal",17,porto,v.n.gaia,portugal
4,5,"farnborough, hants, united kingdom",<NA>,farnborough,hants,united kingdom


### 2. For each book in the *Books* dataset, compute its average rating.

In [47]:
pd.merge(books, book_ratings, on = 'ISBN', how = 'left').groupby('ISBN', as_index=False).agg(Average_Rating=('Book-Rating', 'mean')).head()

# gli 0 vanno mediati?
# mettere colonna book title

,ISBN,Average_Rating
0,0000913154,8.0
1,0001010565,0.0
2,0001046438,9.0
3,0001046713,0.0
4,000104687X,6.0


### 3. For each book in the *GoodBooks* dataset, compute its average rating.

In [23]:
goodbooks[['isbn', 'average_rating']].head()

,isbn,average_rating
0,439023483,4.34
1,439554934,4.44
2,316015849,3.57
3,61120081,4.25
4,743273567,3.89


### 4. Merge together all rows sharing the same book title, author and publisher. We will call the resulting datset `merged books`. The books that have not been merged together will not appear in `merged books`.

In [28]:
merging_books = books[books.duplicated(subset = ['Book-Title', 'Book-Author', 'Publisher'], keep = False)].sort_values(['Book-Title', 'Book-Author', 'Publisher'])
merged_books = merging_books.groupby(['Book-Title', 'Book-Author', 'Publisher'], as_index=False).agg(Count=('ISBN', 'count'))
merged_books.head()

,Book-Title,Book-Author,Publisher,Count
0,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,2
1,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,2
2,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,2
3,101 Famous Poems,Roy J. Cook,McGraw-Hill/Contemporary Books,3
4,15 Houseplants Even You Can't Kill,Joe Elder,Berkley Pub Group,2


### 5. For each book in `merged books` compute its average rating.

The average is computed considering all books in `books` that have been merged.

In [48]:
book_ratings_avg = book_ratings.groupby('ISBN', as_index=True).agg(Average_Rating=('Book-Rating', 'mean'))
merging_book_ratings = pd.merge(merging_books, book_ratings_avg, on = 'ISBN', how = 'left')
merging_book_ratings.head(15)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Average_Rating
0,1565920465,!%@ (A Nutshell handbook),Donnalyn Frey,1994,O'Reilly,http://images.amazon.com/images/P/1565920465.0...,http://images.amazon.com/images/P/1565920465.0...,http://images.amazon.com/images/P/1565920465.0...,0.0
1,1565920317,!%@ (A Nutshell handbook),Donnalyn Frey,1993,O'Reilly,http://images.amazon.com/images/P/1565920317.0...,http://images.amazon.com/images/P/1565920317.0...,http://images.amazon.com/images/P/1565920317.0...,6.0
2,0792276833,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,2000,National Geographic,http://images.amazon.com/images/P/0792276833.0...,http://images.amazon.com/images/P/0792276833.0...,http://images.amazon.com/images/P/0792276833.0...,0.0
3,0792277295,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,2001,National Geographic,http://images.amazon.com/images/P/0792277295.0...,http://images.amazon.com/images/P/0792277295.0...,http://images.amazon.com/images/P/0792277295.0...,3.4
4,1566196256,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,1995,Barnes &amp; Nobles Books,http://images.amazon.com/images/P/1566196256.0...,http://images.amazon.com/images/P/1566196256.0...,http://images.amazon.com/images/P/1566196256.0...,7.25
5,1566196264,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,1995,Barnes &amp; Nobles Books,http://images.amazon.com/images/P/1566196264.0...,http://images.amazon.com/images/P/1566196264.0...,http://images.amazon.com/images/P/1566196264.0...,6.666667
6,0809288311,101 Famous Poems,Roy J. Cook,1984,McGraw-Hill/Contemporary Books,http://images.amazon.com/images/P/0809288311.0...,http://images.amazon.com/images/P/0809288311.0...,http://images.amazon.com/images/P/0809288311.0...,5.0
7,0809288346,101 Famous Poems,Roy J. Cook,1994,McGraw-Hill/Contemporary Books,http://images.amazon.com/images/P/0809288346.0...,http://images.amazon.com/images/P/0809288346.0...,http://images.amazon.com/images/P/0809288346.0...,4.333333
8,0809250969,101 Famous Poems,Roy J. Cook,1985,McGraw-Hill/Contemporary Books,http://images.amazon.com/images/P/0809250969.0...,http://images.amazon.com/images/P/0809250969.0...,http://images.amazon.com/images/P/0809250969.0...,0.0
9,0425073696,15 Houseplants Even You Can't Kill,Joe Elder,1991,Berkley Pub Group,http://images.amazon.com/images/P/0425073696.0...,http://images.amazon.com/images/P/0425073696.0...,http://images.amazon.com/images/P/0425073696.0...,0.0


### 6. For each book in `merged books` compute the minimum and maximum of the average ratings over all corresponding books in the `books` dataset.

Hence for each book in `merged books` we will have exactly two values (a minimum and a maximum)

In [34]:
merging_book_ratings.groupby(['Book-Title', 'Book-Author', 'Publisher'], as_index=False).agg({'Average_Rating' : ['min', 'max']}).head()

Book-Title              Book-Author  \
                                                                                
0                          !%@ (A Nutshell handbook)            Donnalyn Frey   
1  'A Hell of a Place to Lose a Cow': An American...              Tim Brookes   
2  10,000 dreams interpreted: A dictionary of dreams  Gustavus Hindman Miller   
3                                   101 Famous Poems              Roy J. Cook   
4                 15 Houseplants Even You Can't Kill                Joe Elder   

                        Publisher Average_Rating        
                                             min   max  
0                        O'Reilly            0.0   6.0  
1             National Geographic            0.0   3.4  
2       Barnes &amp; Nobles Books       6.666667  7.25  
3  McGraw-Hill/Contemporary Books            0.0   5.0  
4               Berkley Pub Group            0.0   0.0

### 7. For each book in `goodbooks`, compute the list of its authors. Assuming that the number of reviews with a text (column `work_text_reviews_count`) is split equally among all authors, find for each authors the total number of reviews with a text. We will call this quantity the *shared number of reviews with a text*.

In [35]:
goodbooks['authors_splitted'] = goodbooks['authors'].str.split(',')
goodbooks['shared_number_of_reviews_with_a_text'] = goodbooks['work_text_reviews_count'] / goodbooks['authors_splitted'].apply(len)

goodbooks_exploded = goodbooks.explode('authors_splitted')
goodbooks_exploded.groupby(['authors_splitted'], as_index=False).sum('shared_number_of_reviews_with_a_text').head() [['authors_splitted', 'shared_number_of_reviews_with_a_text']]

,authors_splitted,shared_number_of_reviews_with_a_text
0,Alan R. Clarke,27890.500000
1,B.M. Mooyaart-Doubleday,6941.666667
2,Bernard Knox,1620.200000
3,CelÃ¢l Ãster,15172.666667
4,David J. Skal,5754.333333


### 8. For each year of publication, determine the author that has the largest value of the shared number of reviews with a text.

In [36]:
temp = goodbooks_exploded.groupby(['original_publication_year', 'authors_splitted'], as_index=False).sum()
idx = temp.groupby(['original_publication_year'])['shared_number_of_reviews_with_a_text'].transform(max) == temp['shared_number_of_reviews_with_a_text']
temp[idx][['original_publication_year', 'shared_number_of_reviews_with_a_text', 'authors_splitted']].head()

#da sistemare in modo che non ripeta anno e numero

,original_publication_year,shared_number_of_reviews_with_a_text,authors_splitted
0,-720,1620.2,Bernard Knox
1,-720,1620.2,E.V. Rieu
2,-720,1620.2,FrÃ©dÃ©ric Mugler
3,-720,1620.2,Robert Fagles
4,-720,1620.2,Homer


### 9. Assuming that there are no errors in the ISBN fields, find the books in both datasets, and compute the difference of average rating according to the ratings and the goodratings datasets

In [37]:
books_goodbooks = pd.merge(books, goodbooks, left_on = 'ISBN', right_on = 'isbn')
books_goodbooks = pd.merge(books_goodbooks, book_ratings, on = 'ISBN', how = 'left').groupby('ISBN', as_index=False).mean()
books_goodbooks['Difference'] = books_goodbooks['Book-Rating']/2 - books_goodbooks['average_rating']
books_goodbooks.rename(columns={'Book-Rating': 'Book_Average_Rating', 'average_rating': 'Goodbook_Average_Rating'}, inplace=True)
books_goodbooks[['ISBN','Book_Average_Rating', 'Goodbook_Average_Rating', 'Difference']]

#usando average-rating
#da rinominare colonne e da mettere tutto nella stessa scala (5 dei goodbooks e 10 dei books)

,ISBN,Book_Average_Rating,Goodbook_Average_Rating,Difference
0,014028009X,3.751852,3.75,-1.874074
1,014038572X,4.460674,4.06,-1.829663
2,043965548X,3.533333,4.53,-2.763333
3,1400032717,4.813187,3.85,-1.443407
4,1594480001,2.4,4.26,-3.06


In [123]:
books_goodbooks = pd.merge(books, goodbooks, left_on = 'ISBN', right_on = 'isbn')
books_goodbooks = pd.merge(books_goodbooks, book_ratings, on = 'ISBN', how = 'left')
books_goodbooks = pd.merge(books_goodbooks, goodbooks_ratings, on = 'book_id', how = 'left').groupby('ISBN', as_index=False).agg(Book_Average_Rating=('Book-Rating', 'mean'), Goodbook_Average_Rating=('rating', 'mean'))
books_goodbooks['Difference'] = books_goodbooks['Book_Average_Rating']/2 - books_goodbooks['Goodbook_Average_Rating']
books_goodbooks

# usando goodbooks rating
#differenza dei scala voti 

,ISBN,Book_Average_Rating,Goodbook_Average_Rating,Difference
0,014028009X,3.751852,NaN,NaN
1,014038572X,4.460674,NaN,NaN
2,043965548X,3.533333,5.0,-3.233333
3,1400032717,4.813187,NaN,NaN
4,1594480001,2.4,NaN,NaN


### 10. Split the users dataset according to the age. One dataset contains the users with unknown age, one with age 0-14, one with age 15-24, one with age 25-34, and so on.

In [41]:
users.groupby(pd.cut(users['Age'], np.arange(14, 150, 10))).count()

#da fare
#come trattare le etá sbagliate (> 100 anni)?
#come ultimo bin affidabile terrei 84-94

,User-ID,Location,Age,City,Region,Country
Age,,,,,,
"(14, 24]",40001,40001,40001,40001,40001,40001
"(24, 34]",50767,50767,50767,50767,50766,50766
"(34, 44]",32690,32690,32690,32690,32690,32690
"(44, 54]",23152,23152,23152,23152,23152,23152
"(54, 64]",12493,12493,12493,12493,12493,12493
"(64, 74]",3596,3596,3596,3596,3596,3595
"(74, 84]",615,615,615,615,615,615
"(84, 94]",83,83,83,83,83,83
"(94, 104]",278,278,278,278,278,278


In [20]:
#users_UnknownAge_and_Outliers = 



### 11. Find the books that appear only in the goodbooks datasets.

In [140]:
prova = pd.merge(books, goodbooks, left_on = 'ISBN', right_on = 'isbn')
goodbooks[~goodbooks.isin(prova)].dropna()

AssertionError: object

### 12. Assuming that each pair (author, title) identifies a book, for each book find the number of times it appears in the books dataset. Which books appear the most times?

In [133]:
temp = books.groupby(['Book-Author', 'Book-Title'], as_index=False).count()
temp[temp['ISBN'] == temp['ISBN'].max()] [['Book-Title', 'Book-Author', 'ISBN']]

#da rinominare ISBN con count)

,Book-Title,Book-Author,ISBN
150658,Little Women,Louisa May Alcott,21


### 13. Find the author with the highest average rating according to the goodbooks datasets.

In [132]:
prova = goodbooks_exploded.groupby('authors_splitted', as_index=False).mean('average_rating')
prova.loc[prova['average_rating'] == prova['average_rating'].max()] [['authors_splitted', 'average_rating']]

,authors_splitted,average_rating
32,Rufus Beck,4.53
